# CMS Timeliness of Care - Data Cleaning
---

In [1]:
import pandas as pd
import addfips as af
from uszipcode import SearchEngine
from get_cleaned_cms import cms
from get_ms_la_al import shpsa

In [2]:
# Create a list called files for the string paths to the raw data
files = ['raw_data/2016_Timely and Effective Care - Hospital.csv', 'raw_data/2017_Timely and Effective Care - Hospital.csv',
         'raw_data/2018_Timely and Effective Care - Hospital.csv', 'raw_data/2019_Timely and Effective Care - Hospital.csv',
         'raw_data/2020_Timely_and_Effective_Care-Hospital.csv', 'raw_data/2021_Timely_and_Effective_Care-Hospital.csv',
         'raw_data/2022_Timely_and_Effective_Care-Hospital.csv', 'raw_data/2023_Timely_and_Effective_Care-Hospital.csv']

# Create an empty list called dfs to hold the dataframes, then read the .csvs and append to dfs
dfs = []
for file in files:
    dfs.append(pd.read_csv(file, encoding = 'iso-8859-1'))

# Union all dataframes in dfs into one dataframe called toc
toc = pd.concat(dfs, ignore_index = True)

In [3]:
# Get info about dataframe
toc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773480 entries, 0 to 773479
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Year           773480 non-null  int64 
 1   Quarter        773480 non-null  object
 2   Facility ID    773480 non-null  object
 3   Facility Name  773480 non-null  object
 4   Address        773480 non-null  object
 5   City           773480 non-null  object
 6   State          773480 non-null  object
 7   ZIP Code       773480 non-null  int64 
 8   County Name    772475 non-null  object
 9   Phone Number   773480 non-null  object
 10  Condition      773480 non-null  object
 11  Measure ID     773480 non-null  object
 12  Measure Name   773480 non-null  object
 13  Score          773479 non-null  object
 14  Sample         735442 non-null  object
 15  Footnote       493298 non-null  object
 16  Start Date     773480 non-null  object
 17  End Date       773480 non-null  object
dtypes: i

#### Drop unnecessary columns

In [4]:
# Identify columns to be dropped
cols_to_drop = ['Address', 'Phone Number', 'Footnote', 'Start Date', 'End Date']

toc = toc.drop(columns = cols_to_drop)

In [5]:
# Replace state abbrevations in State column with actual state names for states of interest
state_mapping = {
    'AL': 'Alabama',
    'LA': 'Louisiana',
    'MS': 'Mississippi'
}

toc['State'] = toc['State'].replace(state_mapping)

#### I want to add a column named `Common State County FIPS Code` to the `toc` dataframe.  Use the *uszipcode* library to get county name, latitude, & longitude.  Then use the *addfips* library to get the county FIPS code based on the state and county name.

In [6]:
# Get the unique ZIP codes in the toc dataframe
zips = toc['ZIP Code'].unique()

# Convert zips to a list of string ZIP codes (because uszipcode library searches based on a string)
zips = zips.astype(str).tolist()

#### Get state, county name, latitude, and longitude for each ZIP code.

In [7]:
# Create a search engine
search = SearchEngine()

# Create empty dictionary to store the state, county, latitude, & longitude for each ZIP code
zip_map = {}

# Look up each ZIP code, and if that ZIP is in the uszipcode database, add relevant info to zip_map
for zip_code in zips:
    info = search.by_zipcode(zip_code)
    if info:
        zip_map[zip_code] = {
            'State': info.state,
            'County': info.county,
            'Latitude': info.lat,
            'Longitude': info.lng
        }

#### Get the county FIPS code for each ZIP code.

In [8]:
# Initialize AddFIPS object called fips_tool
fips_tool = af.AddFIPS()

# For each ZIP code in zip_map, get the county FIPS code
for zip_code in zip_map:
    county = zip_map[zip_code]['County']
    state = zip_map[zip_code]['State']
    county_fips = fips_tool.get_county_fips(county, state)
    # Add the FIPS code to that ZIP code's dictionary in zip_map
    zip_map[zip_code]['Common State County FIPS Code'] = county_fips

#### Add county name, county FIPS code, latitude, & longitude into `toc` dataframe.

In [9]:
# Define a function called lookup that retrieves a value associated with a specific key for each ZIP code in zip_map
def lookup(zip_code, key):
    return zip_map[zip_code][key]

# Define the keys that will be added as columns to cms
keys = ['County', 'Common State County FIPS Code', 'Latitude', 'Longitude']

# For each key, create a new column in cms, & use the apply method to store the evaluated result of lookup() as the column values
for key in keys:
    toc[key] = toc['ZIP Code'].apply(lambda x: lookup(str(x), key))

In [10]:
# Get shape of toc dataframe
toc.shape

(773480, 17)

#### Drop all rows from measures I won't be exploring.

In [11]:
# Get all measures in toc dataframe
toc['Measure ID'].unique()

array(['ED_1b', 'ED_2b', 'EDV', 'IMM_2', 'IMM_3_OP_27_FAC_ADHPCT', 'OP_1',
       'OP_18b', 'OP_2', 'OP_20', 'OP_21', 'OP_22', 'OP_23', 'OP_29',
       'OP_30', 'OP_31', 'OP_3b', 'OP_4', 'OP_5', 'PC_01', 'STK_4',
       'VTE_5', 'VTE_6', 'OP_18c', 'OP_33', 'SEP_1', 'IMM_3',
       'SEP_SH_3HR', 'SEP_SH_6HR', 'SEV_SEP_3HR', 'SEV_SEP_6HR',
       'HCP_COVID_19', 'ED_2_Strata_1', 'ED_2_Strata_2',
       'SAFE_USE_OF_OPIOIDS', 'STK_02', 'STK_03', 'STK_05', 'STK_06',
       'VTE_1', 'VTE_2'], dtype=object)

In [12]:
# Define measures of interest
measures_to_keep = ['OP_4', 'OP_18b', 'OP_20', 'OP_21', 'OP_22', 'ED_2b']

In [13]:
# Drop irrelevant rows
toc = toc[toc['Measure ID'].isin(measures_to_keep)]

In [14]:
# Check values in Measure ID column
toc['Measure ID'].value_counts()

OP_18b    38037
OP_22     38037
ED_2b     23910
OP_20     14412
OP_21     14412
OP_4      14412
Name: Measure ID, dtype: int64

In [15]:
# Get names of each measure
toc['Measure Name'].unique()

array(['ED2', 'OP 18', 'Door to diagnostic eval',
       'Median time to pain med', 'Left before being seen',
       'Aspirin at Arrival',
       'Average (median) time patients spent in the emergency department, after the doctor decided to admit them as an inpatient before leaving the emergency department for their inpatient room A lower number of minutes is better',
       'Average (median) time patients spent in the emergency department before leaving from the visit A lower number of minutes is better'],
      dtype=object)

In [16]:
# Replace measure strings in Measure Name column with more intuitive names
measure_mapping = {
    'OP 18': 'Median Time Spent in ED Before Leaving',
    'Door to diagnostic eval': 'Median Time Spent in ED Before Seen by Health Professional',
    'Median time to pain med': 'Median Time to Pain Medicine',
    'Left before being seen': '% of Patients Left Before Being Seen',
    'Aspirin at Arrival': 'Median Time to Pain Medicine',
    'Average (median) time patients spent in the emergency department before leaving from the visit A lower number of minutes is better': 'Median Time Spent in ED Before Leaving',
    'ED2': 'Median Admit Decision Time to ED Departure Time As Inpatient',
    'Average (median) time patients spent in the emergency department, after the doctor decided to admit them as an inpatient before leaving the emergency department for their inpatient room A lower number of minutes is better': 'Median Admit Decision Time to ED Departure Time As Inpatient'
}

toc['Measure Name'] = toc['Measure Name'].replace(measure_mapping)

In [17]:
# Check that mapping worked
toc['Measure Name'].value_counts()

Median Time Spent in ED Before Leaving                          38037
% of Patients Left Before Being Seen                            38037
Median Time to Pain Medicine                                    28824
Median Admit Decision Time to ED Departure Time As Inpatient    23910
Median Time Spent in ED Before Seen by Health Professional      14412
Name: Measure Name, dtype: int64

#### Clean the `Score` column.

In [18]:
# Drop rows where Score is Not Available
toc = toc[toc['Score'] != 'Not Available']

In [19]:
# Check the datatype of the column
toc['Score'].dtype

dtype('O')

In [20]:
# Change the datatype to float
toc['Score'] = toc['Score'].astype('float')

# Check to see that the conversion worked
toc['Score'].dtype

dtype('float64')

#### Clean the `Facility ID` column.

In [21]:
# Convert all values in Facility ID column to string datatype
toc['Facility ID'] = toc['Facility ID'].astype(str)

#### Only keep rows from acute care hospitals.  (Use `cms` dataframe to help with this, since there's no Hospital Type column in the Timeliness of Care datasets.)

In [22]:
# Define the hospital type of interest
acute_types = ['Acute Care Hospitals', 'Acute Care - Veterans Administration', 'Acute Care - Department of Defense']

# Subset the cms dataframe, keeping only rows from acute care hospitals
acms = cms[cms['Hospital Type'].isin(acute_types)]

acute_facilities = acms['Facility ID'].unique()

In [23]:
# Get current shape of toc
toc.shape

(108230, 17)

In [24]:
# Only keep the rows in toc from acute care facilities
toc = toc[toc['Facility ID'].isin(acute_facilities)]

In [25]:
# Get the shape of toc after dropping rows
toc.shape

(5781, 17)

In [26]:
# Add a column to toc dataframe for Hospital Ownership
for index, row in toc.iterrows():
    facility_id = row['Facility ID']
    matching_row = acms.loc[acms['Facility ID'] == facility_id]
    toc.at[index, 'Hospital Ownership'] = matching_row['Hospital Ownership'].values[0]

In [27]:
# Export existing toc dataframe to csv
# toc.to_csv('cleaned_data/all_states_toc.csv', index = False)

#### Keep only rows from Alabama, Mississippi, & Louisiana.

In [28]:
# Get all state names
toc['State'].unique()

array(['Alabama', 'Louisiana', 'Mississippi'], dtype=object)

In [29]:
# Drop all states except Alabama, Mississippi, & Louisiana
states = ['Alabama', 'Mississippi', 'Louisiana']

toc = toc[toc['State'].isin(states)]

In [30]:
# Check that the subsetting worked
toc['State'].unique()

array(['Alabama', 'Louisiana', 'Mississippi'], dtype=object)

#### Create a column called County HPSA Status, where a hospital in a HPSA-designated county has the value Shortage Area, and a hospital in a county without a HPSA-designation has the value Non-Shortage Area.

In [31]:
# Get HPSA-designated counties from shpsa dataframe
hpsa_counties = pd.Series(shpsa['Common State County FIPS Code'].unique())

In [32]:
# Create a column County HPSA Status in the toc dataframe with default values of Non-Shortage Area
toc['County HPSA Status'] = 'Non-Shortage Area'

# Check that the column was added with the correct default values
toc['County HPSA Status'].value_counts()

Non-Shortage Area    5781
Name: County HPSA Status, dtype: int64

In [33]:
# Update the County HPSA Status to Shortage Area for hospitals in shortage areas
toc.loc[toc['Common State County FIPS Code'].isin(hpsa_counties), 'County HPSA Status'] = 'Shortage Area'

In [34]:
# Check the counts of Shortage Area vs. Non-Shortage area in the new column
toc['County HPSA Status'].value_counts()

Non-Shortage Area    3638
Shortage Area        2143
Name: County HPSA Status, dtype: int64

#### Left join `toc` with `shpsa` based on county FIPS code, and only join relevant columns from `shpsa`.

In [35]:
# Identify columns of interest from shpsa, then left join cms with shpsa based on county FIPS code
join_cols = ['Common State County FIPS Code', 'Designation Type', 'HPSA Score', 'HPSA FTE',
             'HPSA Designation Population', '% of Population Below 100% Poverty', 'HPSA Formal Ratio',
             'Rural Status', 'HPSA Provider Ratio Goal', 'HPSA Shortage']

toc = toc.merge(shpsa[join_cols], on = 'Common State County FIPS Code', how = 'left')

#### If info exists in `ahrq` dataframe, add number of hospital beds and number of doctors for a specific hospital to the `toc` dataframe.

In [36]:
# Import data from AHRQ into dataframe called ahrq
ahrq = pd.read_excel('raw_data/chsp-compendium-2021.xlsx')

In [37]:
toc.columns

Index(['Year', 'Quarter', 'Facility ID', 'Facility Name', 'City', 'State',
       'ZIP Code', 'County Name', 'Condition', 'Measure ID', 'Measure Name',
       'Score', 'Sample', 'County', 'Common State County FIPS Code',
       'Latitude', 'Longitude', 'Hospital Ownership', 'County HPSA Status',
       'Designation Type', 'HPSA Score', 'HPSA FTE',
       'HPSA Designation Population', '% of Population Below 100% Poverty',
       'HPSA Formal Ratio', 'Rural Status', 'HPSA Provider Ratio Goal',
       'HPSA Shortage'],
      dtype='object')

In [38]:
# Only keep rows from MS, AL, & LA
ahrq = ahrq[ahrq['health_sys_state'].isin(['MS', 'AL', 'LA'])]

In [39]:
# Check that subsetting worked
ahrq['health_sys_state'].unique()

array(['LA', 'MS', 'AL'], dtype=object)

In [40]:
# Create hospital mapping, where key is the Facility Name from toc, and value is the corresponding health_sys_name from ahrq
# Mapping was validated by eye using Excel, checking that the was a match in health system city in AHRQ and CMS .csv files
hosp_mapping = {
    'DCH REGIONAL MEDICAL CENTER': 'DCH Health System',
    'EAST ALABAMA MEDICAL CENTER': 'East Alabama Medical Center',
    'HUNTSVILLE HOSPITAL': 'Huntsville Hospital Health System',
    'MOBILE INFIRMARY MEDICAL CENTER': 'Infirmary Health System',
    'SOUTHEAST HEALTH MEDICAL CENTER': 'Southeast Health',
    'UAB CALLAHAN EYE HOSPITAL AUTHORITY': 'UAB Health System',
    'UNIVERSITY OF SOUTH ALABAMA MEDICAL CENTER': 'University of South Alabama Hospitals',
    'BATON ROUGE GENERAL MEDICAL CENTER': 'Baton Rouge General Health System',
    'OUR LADY OF THE LAKE REGIONAL MEDICAL CENTER': 'Franciscan Missionaries of Our Lady Health System',
    'UNIVERSITY MEDICAL CENTER NEW ORLEANS': 'LCMC Health System',
    'TOURO INFIRMARY': 'LCMC Health System',
    'NEW ORLEANS EAST HOSPITAL': 'LCMC Health System',
    'LAKE CHARLES MEMORIAL HOSPITAL': 'Lake Charles Memorial Health System',
    'NORTH OAKS MEDICAL CENTER, L L C': 'North Oaks Health System',
    'OCHSNER CLINIC FOUNDATION': 'Ochsner Health System',
    'OCHSNER MEDICAL CENTER': 'Ochsner Health System',
    'OCHSNER MEDICAL CENTER - BATON ROUGE': 'Ochsner Health System',
    'OCHSNER MEDICAL CENTER - NORTHSHORE, L L C': 'Ochsner Health System',
    'OCHSNER MEDICAL CENTER KENNER': 'Ochsner Health System',
    'OCHSNER LSU HEALTH MONROE': 'Ochsner Health System',
    'OCHSNER LSU HEALTH SHREVEPORT': 'Ochsner Health System',
    'OCHSNER ST MARY': 'Ochsner Health System',
    'OCHSNER LAFAYETTE GENERAL MEDICAL CENTER': 'Ochsner Health System',
    'OCHSNER UNIVERSITY HOSPITAL AND CLINICS': 'Ochsner Health System',
    'OCHSNER AMERICAN LEGION HOSPITAL': 'Ochsner Health System',
    'SLIDELL MEMORIAL HOSPITAL': 'Slidell Memorial Hospital',
    'WILLIS KNIGHTON MEDICAL CENTER': 'Willis Knighton Health System',
    'WILLIS KNIGHTON MEDICAL CENTER, INC': 'Willis Knighton Health System',
    'ANDERSON REGIONAL MEDICAL CENTER': 'Anderson Regional Health System',
    'ANDERSON REGIONAL MEDICAL CTR': 'Anderson Regional Health System',
    'DELTA HEALTH-NORTHWEST REGIONAL': 'Delta Health System',
    'DELTA HEALTH- THE MEDICAL CENTER': 'Delta Health System',
    'DELTA HEALTH SYSTEM - THE MEDICAL CENTER': 'Delta Health System',
    'DELTA HEALTH - HIGHLAND  HILLS': 'Delta Health System',
    'FORREST GENERAL HOSPITAL': 'Forrest Health',
    'MARION GENERAL HOSPITAL': 'Forrest Health',
    'MEMORIAL HOSPITAL AT GULFPORT': 'Memorial Hospital at Gulfport',
    'NORTH MISSISSIPPI MEDICAL CENTER': 'North Mississippi Health Services',
    'NORTH MISSISSIPPI MEDICAL CENTER-WEST POINT': 'North Mississippi Health Services',
    'OCHSNER RUSH HOSPITAL': 'Rush Health Systems',
    'RUSH FOUNDATION HOSPITAL': 'Rush Health Systems',
    'SINGING RIVER HEALTH SYSTEM': 'Singing River Health System',
    'SINGING RIVER GULFPORT': 'Singing River Health System',
    'SINGING RIVER HOSPITAL': 'Singing River Health System',
    'UNIVERSITY OF MISSISSIPPI MEDICAL CENTER- GRENADA': 'The University of Mississippi Medical Center',
    'UNIVERSITY OF MISSISSIPPI MED CENTER': 'The University of Mississippi Medical Center',
    'SOUTH CENTRAL REG MED CTR': 'South Central Regional Medical Center Health System'
}

# Define columns in ahrq that I want to keep (keys), and map to final column name (values)
ahrq_cols = {
    'total_mds': 'Total MDs in System',
    'prim_care_mds': 'Total Primary Care MDs in System',
    'hosp_cnt': 'Total Hospitals in System',
    'acutehosp_cnt': 'Total Acute Care Hospitals in System',
    'sys_beds': 'Total Beds in System',
    'sys_res': 'Total Residents in System'
}

#### Iterate through each `Facility Name` in `toc` dataframe, and append new columns from `ahrq` if there is a match in `hosp_mapping`.

In [41]:
for col in ahrq_cols.keys():
    for i in toc.index:
        facility = toc.loc[i, 'Facility Name']  # Accessing 'Facility Name' column in 'toc'
        if facility in hosp_mapping:
            matching_health_sys = ahrq.loc[ahrq['health_sys_name'] == hosp_mapping[facility], col]
            toc.at[i, ahrq_cols[col]] = matching_health_sys.values[0] if not matching_health_sys.empty else None

In [42]:
# Get info for final toc dataframe
toc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5781 entries, 0 to 5780
Data columns (total 34 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Year                                  5781 non-null   int64  
 1   Quarter                               5781 non-null   object 
 2   Facility ID                           5781 non-null   object 
 3   Facility Name                         5781 non-null   object 
 4   City                                  5781 non-null   object 
 5   State                                 5781 non-null   object 
 6   ZIP Code                              5781 non-null   int64  
 7   County Name                           5781 non-null   object 
 8   Condition                             5781 non-null   object 
 9   Measure ID                            5781 non-null   object 
 10  Measure Name                          5781 non-null   object 
 11  Score            

In [44]:
# Export final toc dataframe (with only Mississippi, Alabama, & Louisiana data) to csv
# toc.to_csv('cleaned_data/toc.csv', index = False)